In [1]:
# File location and type
file_location = "/FileStore/tables/CC_GENERAL-f7a37.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
CCdf = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

CCdf.count() #8950

Out[ 9 ]: 8950

In [2]:
#dir()
CCdf.printSchema()

root
-- CUST_ID: string (nullable = true)
-- BALANCE: double (nullable = true)
-- BALANCE_FREQUENCY: double (nullable = true)
-- PURCHASES: double (nullable = true)
-- ONEOFF_PURCHASES: double (nullable = true)
-- INSTALLMENTS_PURCHASES: double (nullable = true)
-- CASH_ADVANCE: double (nullable = true)
-- PURCHASES_FREQUENCY: double (nullable = true)
-- ONEOFF_PURCHASES_FREQUENCY: double (nullable = true)
-- PURCHASES_INSTALLMENTS_FREQUENCY: double (nullable = true)
-- CASH_ADVANCE_FREQUENCY: double (nullable = true)
-- CASH_ADVANCE_TRX: integer (nullable = true)
-- PURCHASES_TRX: integer (nullable = true)
-- CREDIT_LIMIT: double (nullable = true)
-- PAYMENTS: double (nullable = true)
-- MINIMUM_PAYMENTS: double (nullable = true)
-- PRC_FULL_PAYMENT: double (nullable = true)
-- TENURE: integer (nullable = true)

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
CCdf=CCdf.dropna()
CCdf.count() #8636

Out[ 13 ]: 8636

In [4]:
fac_coords_df = (CCdf                 .select('BALANCE','BALANCE_FREQUENCY','PURCHASES','ONEOFF_PURCHASES','INSTALLMENTS_PURCHASES','CASH_ADVANCE','PURCHASES_FREQUENCY','ONEOFF_PURCHASES_FREQUENCY','PURCHASES_INSTALLMENTS_FREQUENCY','CASH_ADVANCE_FREQUENCY','CASH_ADVANCE_TRX','PURCHASES_TRX','CREDIT_LIMIT','PAYMENTS','MINIMUM_PAYMENTS','PRC_FULL_PAYMENT','TENURE'))

In [5]:
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=['BALANCE','BALANCE_FREQUENCY','PURCHASES','ONEOFF_PURCHASES','INSTALLMENTS_PURCHASES','CASH_ADVANCE','PURCHASES_FREQUENCY','ONEOFF_PURCHASES_FREQUENCY','PURCHASES_INSTALLMENTS_FREQUENCY','CASH_ADVANCE_FREQUENCY','CASH_ADVANCE_TRX','PURCHASES_TRX','CREDIT_LIMIT','PAYMENTS','MINIMUM_PAYMENTS','PRC_FULL_PAYMENT','TENURE'], outputCol="features")
new_df = vecAssembler.transform(fac_coords_df)
new_df.show(1)

+---------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+----------+----------------+----------------+------+--------------------+
 BALANCE|BALANCE_FREQUENCY|PURCHASES|ONEOFF_PURCHASES|INSTALLMENTS_PURCHASES|CASH_ADVANCE|PURCHASES_FREQUENCY|ONEOFF_PURCHASES_FREQUENCY|PURCHASES_INSTALLMENTS_FREQUENCY|CASH_ADVANCE_FREQUENCY|CASH_ADVANCE_TRX|PURCHASES_TRX|CREDIT_LIMIT| PAYMENTS|MINIMUM_PAYMENTS|PRC_FULL_PAYMENT|TENURE| features|
+---------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+----------+----------------+----------------+------+--------------------+
40.900749| 0.818182| 95.4| 0.0| 95.4| 0.0| 0.166667| 0.0| 0.083333| 0.0| 0| 2| 1000.0|201.802084| 139.509787| 0.0| 12|[40.900749,0.8181...|
+---------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+----------+----------------+----------------+------+--------------------+
only showing top 1 row

In [6]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans(k=5, seed=1)  # 5 clusters here
Kmeans_model = kmeans.fit(new_df.select('features'))

In [7]:
clusterPredicted = Kmeans_model.transform(new_df)

In [8]:
cp = clusterPredicted.drop('features').collect()
display(cp)

BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE,prediction
40.900749,0.818182,95.4,0.0,95.4,0.0,0.166667,0.0,0.083333,0.0,0,2,1000.0,201.802084,139.509787,0.0,12,2
3202.467416,0.909091,0.0,0.0,0.0,6442.945483,0.0,0.0,0.0,0.25,4,0,7000.0,4103.032597,1072.340217,0.222222,12,3
2495.148862,1.0,773.17,773.17,0.0,0.0,1.0,1.0,0.0,0.0,0,12,7500.0,622.066742,627.284787,0.0,12,0
817.714335,1.0,16.0,16.0,0.0,0.0,0.083333,0.083333,0.0,0.0,0,1,1200.0,678.334763,244.791237,0.0,12,2
1809.828751,1.0,1333.28,0.0,1333.28,0.0,0.666667,0.0,0.583333,0.0,0,8,1800.0,1400.05777,2407.246035,0.0,12,2
627.260806,1.0,7091.01,6402.63,688.38,0.0,1.0,1.0,1.0,0.0,0,64,13500.0,6354.314328,198.065894,1.0,12,0
1823.652743,1.0,436.2,0.0,436.2,0.0,1.0,0.0,1.0,0.0,0,12,2300.0,679.065082,532.03399,0.0,12,2
1014.926473,1.0,861.49,661.49,200.0,0.0,0.333333,0.083333,0.25,0.0,0,5,7000.0,688.278568,311.963409,0.0,12,0
152.225975,0.545455,1281.6,1281.6,0.0,0.0,0.166667,0.166667,0.0,0.0,0,3,11000.0,1164.770591,100.302262,0.0,12,0
1293.124939,1.0,920.12,0.0,920.12,0.0,1.0,0.0,1.0,0.0,0,12,1200.0,1083.301007,2172.697765,0.0,12,2


In [9]:
from pyspark.ml.evaluation import ClusteringEvaluator
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(clusterPredicted)
print("Silhouette with squared euclidean distance = " + str(silhouette)) 

Silhouette with squared euclidean distance = 0.510790502373792

In [10]:
from pyspark.ml.evaluation import ClusteringEvaluator
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(clusterPredicted)
print("Silhouette with squared euclidean distance = " + str(silhouette)) 

Silhouette with squared euclidean distance = 0.5324592353317451

In [11]:
from pyspark.ml.evaluation import ClusteringEvaluator
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(clusterPredicted)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.4775078414249911

In [12]:
ClusterCenter = Kmeans_model.clusterCenters()
print("Cluster Centers: ")
for i in ClusterCenter:
    print(ClusterCenter)

Cluster Centers: 
[array([ 1.78596417e+03, 9.16059657e-01, 1.65108319e+03,
 1.04604320e+03, 6.05439628e+02, 7.48256876e+02,
 6.03294372e-01, 3.54118143e-01, 4.20826736e-01,
 1.11151121e-01, 2.49893025e+00, 2.28831836e+01,
 7.60521453e+03, 2.09954763e+03, 6.74034743e+02,
 2.05401408e-01, 1.17843389e+01]), array([ 4.45124178e+03, 9.85645947e-01, 1.16328816e+03,
 1.68416053e+02, 9.94872105e+02, 1.00375484e+03,
 4.99401868e-01, 4.90430000e-02, 4.70295053e-01,
 8.89154474e-02, 2.97368421e+00, 2.24210526e+01,
 4.68815789e+03, 1.63663139e+03, 2.73619541e+04,
 0.00000000e+00, 1.19210526e+01]), array([ 8.60542387e+02, 8.75827779e-01, 5.25050891e+02,
 2.55849309e+02, 2.69506461e+02, 5.23599269e+02,
 4.57225722e-01, 1.35709853e-01, 3.49915473e-01,
 1.14952012e-01, 2.43228880e+00, 9.83599047e+00,
 2.29990603e+03, 9.56568281e+02, 5.54209285e+02,
 1.52520667e-01, 1.14062672e+01]), array([ 6.11171197e+03, 9.61939189e-01, 1.14886582e+03,
 6.62400641e+02, 4.86558426e+02, 5.10041471e+03,
 3.96957777e-01, 2.04752635e-01, 2.98293483e-01,
 3.99258805e-01, 1.23008357e+01, 1.74679666e+01,
 1.03491010e+04, 5.00633259e+03, 2.21947062e+03,
 3.89497117e-02, 1.16225627e+01]), array([ 4.66329910e+03, 9.43868930e-01, 1.46833978e+04,
 1.04785200e+04, 4.20513360e+03, 3.25002138e+03,
 8.66085256e-01, 7.11733628e-01, 6.99136709e-01,
 1.30813930e-01, 6.52325581e+00, 1.07906977e+02,
 1.30232558e+04, 1.89230016e+04, 2.68911700e+03,
 4.12230674e-01, 1.19651163e+01])]
[array([ 1.78596417e+03, 9.16059657e-01, 1.65108319e+03,
 1.04604320e+03, 6.05439628e+02, 7.48256876e+02,
 6.03294372e-01, 3.54118143e-01, 4.20826736e-01,
 1.11151121e-01, 2.49893025e+00, 2.28831836e+01,
 7.60521453e+03, 2.09954763e+03, 6.74034743e+02,
 2.05401408e-01, 1.17843389e+01]), array([ 4.45124178e+03, 9.85645947e-01, 1.16328816e+03,
 1.68416053e+02, 9.94872105e+02, 1.00375484e+03,
 4.99401868e-01, 4.90430000e-02, 4.70295053e-01,
 8.89154474e-02, 2.97368421e+00, 2.24210526e+01,
 4.68815789e+03, 1.63663139e+03, 2.73619541e+04,
 0.00000000e+00, 1.19210526e+01]), array([ 8.60542387e+02, 8.75827779e-01, 5.25050891e+02,
 2.55849309e+02, 2.69506461e+02, 5.23599269e+02,
 4.57225722e-01, 1.35709853e-01, 3.49915473e-01,
 1.14952012e-01, 2.43228880e+00, 9.83599047e+00,
 2.29990603e+03, 9.56568281e+02, 5.54209285e+02,
 1.52520667e-01, 1.14062672e+01]), array([ 6.11171197e+03, 9.61939189e-01, 1.14886582e+03,
 6.62400641e+02, 4.86558426e+02, 5.10041471e+03,
 3.96957777e-01, 2.04752635e-01, 2.98293483e-01,
 3.99258805e-01, 1.23008357e+01, 1.74679666e+01,
 1.03491010e+04, 5.00633259e+03, 2.21947062e+03,
 3.89497117e-02, 1.16225627e+01]), array([ 4.66329910e+03, 9.43868930e-01, 1.46833978e+04,
 1.04785200e+04, 4.20513360e+03, 3.25002138e+03,
 8.66085256e-01, 7.11733628e-01, 6.99136709e-01,
 1.30813930e-01, 6.52325581e+00, 1.07906977e+02,
 1.30232558e+04, 1.89230016e+04, 2.68911700e+03,
 4.12230674e-01, 1.19651163e+01])]
[array([ 1.78596417e+03, 9.16059657e-01, 1.65108319e+03,
 1.04604320e+03, 6.05439628e+02, 7.48256876e+02,
 6.03294372e-01, 3.54118143e-01, 4.20826736e-01,
 1.11151121e-01, 2.49893025e+00, 2.28831836e+01,
 7.60521453e+03, 2.09954763e+03, 6.74034743e+02,
 2.05401408e-01, 1.17843389e+01]), array([ 4.45124178e+03, 9.85645947e-01, 1.16328816e+03,
 1.68416053e+02, 9.94872105e+02, 1.00375484e+03,
 4.99401868e-01, 4.90430000e-02, 4.70295053e-01,
 8.89154474e-02, 2.97368421e+00, 2.24210526e+01,
 4.68815789e+03, 1.63663139e+03, 2.73619541e+04,
 0.00000000e+00, 1.19210526e+01]), array([ 8.60542387e+02, 8.75827779e-01, 5.25050891e+02,
 2.55849309e+02, 2.69506461e+02, 5.23599269e+02,
 4.57225722e-01, 1.35709853e-01, 3.49915473e-01,
 1.14952012e-01, 2.43228880e+00, 9.83599047e+00,
 2.29990603e+03, 9.56568281e+02, 5.54209285e+02,
 1.52520667e-01, 1.14062672e+01]), array([ 6.11171197e+03, 9.61939189e-01, 1.14886582e+03,
 6.62400641e+02, 4.86558426e+02, 5.10041471e+03,
 3.96957777e-01, 2.04752635e-01, 2.98293483e-01,
 3.99258805e-01, 1.23008357e+01, 1.74679666e+01,
 1.03491010e+04, 5.00633259e+03, 2.21947062e+